In [ ]:
!git clone https://github.com/NafNawal04/tryolab_football.git
%cd tryolab_football

In [ ]:
!git clone https://github.com/SilvioGiancola/SoccerNetv2-DevKit.git soccernet/SoccerNetv2-DevKit
%cd soccernet/SoccerNetv2-DevKit/Task1-ActionSpotting/CALF

!pip install torch torchvision torchaudio
!pip install SoccerNet

# Install system dependencies
!apt-get update
!apt-get install -y ffmpeg

# Install Python packages
!pip install -q scikit-video
!pip install -q tensorflow==2.3.0
!pip install -q imutils
!pip install -q opencv-python==3.4.11.41
!pip install -q SoccerNet
!pip install -q moviepy
!pip install -q scikit-learn
!pip install -q ffmpy
!pip install -q torch torchvision

In [ ]:
%cd soccernet/SoccerNetv2-DevKit/Task1-ActionSpotting/CALF
from pathlib import Path

preprocessing_path = Path("/content/tryolab_football/soccernet/SoccerNetv2-DevKit/Task1-ActionSpotting/CALF/inference/preprocessing.py")

old_stamp = (
    "            # tryolab_autopatch\n"
    "            copy_end = min(chunk_size - receptive_field, video_size, tmp_timestamps.size(0))\n"
    "            timestamps_long[0:copy_end] = tmp_timestamps[0:copy_end]"
)
new_stamp = (
    "            # tryolab_autopatch\n"
    "            copy_end = max(0, min(chunk_size - receptive_field, video_size, tmp_timestamps.size(0)))\n"
    "            if copy_end:\n"
    "                timestamps_long[0:copy_end] = tmp_timestamps[0:copy_end]"
)

old_stamp_seg = (
    "            # tryolab_autopatch\n"
    "            copy_end = min(chunk_size - receptive_field, video_size, tmp_segmentation.size(0))\n"
    "            segmentation_long[0:copy_end] = tmp_segmentation[0:copy_end]"
)
new_stamp_seg = (
    "            # tryolab_autopatch\n"
    "            copy_end = max(0, min(chunk_size - receptive_field, video_size, tmp_segmentation.size(0)))\n"
    "            if copy_end:\n"
    "                segmentation_long[0:copy_end] = tmp_segmentation[0:copy_end]"
)

text = preprocessing_path.read_text()
text = text.replace(old_stamp, new_stamp)
text = text.replace(old_stamp_seg, new_stamp_seg)
preprocessing_path.write_text(text)
print("Updated preprocessing.py with guarded copy length.")

In [ ]:
import textwrap

dataset_path = Path(
    "/content/tryolab_football/soccernet/SoccerNetv2-DevKit/Task1-ActionSpotting/CALF/inference/dataset.py"
)

target_fn = "def feats2clip(feats, idx):"
new_fn = textwrap.dedent(
    """
    def feats2clip(feats, idx):
        \"\"\"Return a (chunk_size, feature_dim) tensor, zero-padding if idx goes outside range.\"\"\"
        feature_dim = feats.size(1)

        def _pad_and_copy(start_idx, stop_idx, step=1):
            length = max(0, stop_idx - start_idx)
            if length == 0:
                return torch.zeros(0, feature_dim, dtype=feats.dtype)
            pad_left = max(0, -start_idx)
            pad_right = max(0, start_idx + length - feats.size(0))
            start_idx_clamped = max(0, start_idx)
            stop_idx_clamped = min(feats.size(0), start_idx + length)
            clip = feats[start_idx_clamped:stop_idx_clamped:step]
            if pad_left or pad_right or clip.size(0) < length:
                padded = torch.zeros(length, feature_dim, dtype=feats.dtype)
                if clip.size(0) > 0:
                    padded[pad_left:pad_left + clip.size(0)] = clip
                return padded
            return clip

        if isinstance(idx, slice):
            start = idx.start or 0
            stop = idx.stop if idx.stop is not None else feats.size(0)
            step = idx.step or 1
            if step != 1:
                raise ValueError("feats2clip expects step=1 slices.")
            return _pad_and_copy(start, stop, step)

        safe_idx = min(max(idx, 0), feats.size(0) - 1)
        if safe_idx != idx:
            return torch.zeros(feature_dim, dtype=feats.dtype)
        return feats[safe_idx]
    """
)

text = dataset_path.read_text(encoding="utf-8")
if target_fn not in text:
    raise RuntimeError("Could not find feats2clip in dataset.py")

before, after = text.split(target_fn, maxsplit=1)
after = after.split("def ", 1)[-1]
dataset_path.write_text(before + new_fn + "def " + after, encoding="utf-8")
print("Updated feats2clip to clamp/zero-pad out-of-range slices.")



> **In soccer-video-analytics folder, create two folders "models" and "videos". Then in models, add ball.pt file and in videos, add the soccer_possession.mp4 video**



In [ ]:
%cd /content/tryolab_football

In [ ]:
!pip install opencv-python
!pip install numpy
!pip install Pillow
!pip install norfair[video]
!pip install ultralytics

In [ ]:
!python run.py --passes --possession --model models/ball.pt --video videos/short_vid.mp4 --soccernet